In [ ]:
import pickle
import json
import re
import os
import math

1.读取初始数据

In [ ]:
data = []
count = 0
with open('baike_triples.txt', 'r', encoding='utf8') as f:
    d = f.readline()
    while d:
        if count%3000000==0:
            print(count)
        count+=1
        data.append(d.strip().split('\t'))
        d = f.readline()
print(len(data))
data[0]

2.保留元素全为中文的三元组

In [ ]:
all_chinese = re.compile('^[\u4e00-\u9fa5]*$')
new_data = []
for triple in data:
    if bool(re.search(all_chinese, triple[0])):
        if bool(re.search(all_chinese, triple[1])):
            if bool(re.search(all_chinese, triple[2])):
                new_data.append(triple)
len(new_data)

3.实体字典

In [ ]:
entities = {}
count = 0
for can in new_data:
    if count %3000000==0:
        print(count)
    count+=1
    entities[can[0]] = entities.get(can[0], {})
    entities[can[0]][can[1]] = can[2]

In [ ]:
if not os.path.isdir('processed'):
    os.mkdir('processed')
with open('processed/entities.pkl', 'wb') as f:
    pickle.dump(entities, f)

In [ ]:
e_set = set(list(entities.keys()))
with open('dict.txt', 'w', encoding='utf8') as f:
    for e in e_set:
        f.write(e+'\n')

4.获取句子集合

In [ ]:
sentences = []
for can in data:
    if can[1]=='BaiduCARD':
        sentences.append(can[2])
len(sentences)

In [ ]:
sentences[0]

In [ ]:
new_sentences = []
for i in range(len(sentences)):
    tp = sentences[i]
    sen = re.sub(' +', ' ', tp)
    sen = re.sub('[^\u4e00-\u9fa5,\?\!，。？：:！、；\(\)（） ]', '', sen)
    sen = re.split('[\?\!。？！]', sen)
    for s in sen:
        if s!='':
            new_sentences.append(s)
print(len(new_sentences))

5.句子存为100个文件

In [ ]:
if not os.path.isdir('processed/sentences'):
    os.mkdir('processed/sentences')

begin = 0
count = math.ceil(len(new_sentences)*1.0/100)
print(count)
end = min(begin+count, len(new_sentences))
for i in range(100):
    with open('processed/sentences/sen'+str(i), 'wb') as f:
        pickle.dump(new_sentences[begin:end], f)
    begin = end
    end = min(begin+count, len(new_sentences))